In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [34]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score
#import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

import xgboost as xgb


In [4]:
#import the data
data_features_train = pd.read_csv('/content/drive/MyDrive/ml_task2/train_features.csv',index_col='pid').sort_values(by=['pid', 'Time'])
data_features_test = pd.read_csv('/content/drive/MyDrive/ml_task2/test_features.csv',index_col='pid').sort_values(by=['pid', 'Time'])
data_labels_train = pd.read_csv('/content/drive/MyDrive/ml_task2/train_labels.csv', index_col='pid').sort_values(by=['pid'])


vitals = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
tests = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
         'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2']
sepsis = ['LABEL_Sepsis']


# Task 1 & 2 yolo approach

In [44]:
# #task 1&2
# clf = tests + sepsis

# X_12_train_mean = data_features_train.groupby(by='pid', dropna=False).mean()
# X_12_train =X_12_train_mean.drop(['Time'], axis=1).to_numpy()


# X_12_test_mean = data_features_test.groupby(by='pid', dropna=False).mean()
# X_12_test =X_12_test_mean.drop(['Time'], axis=1).to_numpy()



# #y_2_train = data_labels_train[sepsis].to_numpy()
# #dX_train = xgb.DMatrix(X_12_train, label=y_2_train)
# #dX_test = xgb.DMatrix(X_12_test, label=y_2_train)

# param = {'objective': 'binary:logistic',
#          'eval metric': 'auc'}
# epochs = 20

# scores = []
# predictions_12 = np.empty((len(data_features_test.index.unique()),len(clf)))

# m=0

# for test_ in clf:
#   y_2_train = data_labels_train[test_].to_numpy()

#   X_train, X_test, y_train, y_test = train_test_split(X_12_train, y_2_train, test_size=0.33, random_state=1)

#   dX_train = xgb.DMatrix(X_train, label=y_train)
#   dX_test = xgb.DMatrix(X_test, label=y_test)

#   clf_xgb = xgb.train(param, dX_train, epochs)

#   #predictions_12[:,m] = clf_xgb.predict(dX_test)
#   scores.append(roc_auc_score(y_test, clf_xgb.predict(dX_test)))
#   m=m+1

# print(scores)
# #clf_predict = clf_xgb.predict(dX_test)
# print(f'The mean ROC AUC is: {np.mean(scores)}.')

[0.9265571642014852, 0.7964798099938493, 0.7275732532765031, 0.738402828168292, 0.7291763901773541, 0.7997565931865904, 0.8918165549149111, 0.8242919012313402, 0.7703197683060587, 0.9461558814308256, 0.7152071367746338]
The mean ROC AUC is: 0.805976116514713.


In [45]:
#task 1&2
clf = tests + sepsis

X_12_train_mean = data_features_train.groupby(by='pid', dropna=False).mean()
X_12_train =X_12_train_mean.drop(['Time'], axis=1).to_numpy()


X_12_test_mean = data_features_test.groupby(by='pid', dropna=False).mean()
X_12_test =X_12_test_mean.drop(['Time'], axis=1).to_numpy()



#y_2_train = data_labels_train[sepsis].to_numpy()
#dX_train = xgb.DMatrix(X_12_train, label=y_2_train)
#dX_test = xgb.DMatrix(X_12_test, label=y_2_train)

param = {'objective': 'binary:logistic',
         'eval metric': 'auc'}
epochs = 40

scores = []
predictions_12 = np.empty((len(data_features_test.index.unique()),len(clf)))

m=0

for test_ in clf:
  y_2_train = data_labels_train[test_].to_numpy()
  dX_train = xgb.DMatrix(X_12_train, label=y_2_train)
  dX_test = xgb.DMatrix(X_12_test, label=y_2_train)

  clf_xgb = xgb.train(param, dX_train, epochs)

  predictions_12[:,m] = clf_xgb.predict(dX_test)
  scores.append(roc_auc_score(y_2_train, clf_xgb.predict(dX_train)))
  m=m+1

#clf_predict = clf_xgb.predict(dX_test)
print(f'The mean ROC AUC is: {np.mean(scores)}.')

The mean ROC AUC is: 0.947275255345252.


In [52]:
df_12 = pd.DataFrame(predictions_12, index=data_features_test.index.unique(), columns=data_labels_train[clf].columns)

# Feature Engineering

In [17]:
def feature_engineering(df):
    df_pop_mean = df.drop(['Age'], axis=1).mean()
    #construct the features
    
    df_age = df['Age'].groupby(by='pid', dropna=False).mean() #this is a bit hackerboy method lmao
    
    df_mean = df.drop(['Age'], axis=1).groupby(by='pid', dropna=False).mean()
    df_var = df.drop(['Age'], axis=1).groupby(by='pid', dropna=False).var()
    df_min = df.drop(['Age'], axis=1).groupby(by='pid', dropna=False).min()
    df_max = df.drop(['Age'], axis=1).groupby(by='pid', dropna=False).max()
    df_first = df.drop(['Age'], axis=1).groupby(by='pid', dropna=False).first()
    df_last = df.drop(['Age'], axis=1).groupby(by='pid', dropna=False).last()
    df_nan_count = df.drop(['Age'], axis=1).groupby(by='pid', dropna=False).count()
    #df_last_observed_value = df.groupby(by='pid', dropna=False)
    #df_first_observed_value
    
    
    
    #impute missing values
    df_mean.fillna({col:df_pop_mean[col] for col in df.drop(['Age'], axis=1).columns}, inplace=True)
    df_var.fillna({col:df_pop_mean[col] for col in df.drop(['Age'], axis=1).columns}, inplace=True)
    df_min.fillna({col:df_pop_mean[col] for col in df.drop(['Age'], axis=1).columns}, inplace=True)
    df_max.fillna({col:df_pop_mean[col] for col in df.drop(['Age'], axis=1).columns}, inplace=True)
    df_first.fillna({col:df_pop_mean[col] for col in df.drop(['Age'], axis=1).columns}, inplace=True)
    df_last.fillna({col:df_pop_mean[col] for col in df.drop(['Age'], axis=1).columns}, inplace=True)
    #df_last_observed_value.fillna({col:df_pop_mean[col] for col in df.drop(['Age'], axis=1).columns}, inplace=True)
    #df_first_observed_value.fillna({col:df_pop_mean[col] for col in df.drop(['Age'], axis=1).columns}, inplace=True)
    
    df_features = pd.concat([df_age, df_mean.add_suffix('_mean'), df_var.add_suffix('_var'),
                             df_min.add_suffix('_min'), df_max.add_suffix('_max'), df_first.add_suffix('_first'),
                             df_last.add_suffix('_last'), df_nan_count.add_suffix('_nan_count')], axis=1)
    return df_features

In [18]:
#set up features, took most correlated ones for each of 'RRate', 'ABPm', 'SpO2', 'Heartrate'

features_3 = ['Age','Temp', 'Lactate', 'ABPs', 'Chloride', 'Hgb', 'pH', 'ABPd', 'RRate', 'ABPm', 'SpO2', 'Heartrate']

X_3_train = feature_engineering(data_features_train[features_3]).to_numpy()
X_3_test = feature_engineering(data_features_test[features_3]).to_numpy()
y_3_train = data_labels_train[vitals].to_numpy()

In [19]:
# Define a pipeline to search for the best hyperparameters for ridge/randomforest

pipe = Pipeline([('scaler', StandardScaler()), ('regr', RandomForestRegressor())])

# Parameter space:
max_features = ['sqrt']
n_estimators = [500,600,700,800]

param_grid = {'regr':[RandomForestRegressor()],
            'regr__max_features':max_features,
            'regr__n_estimators':n_estimators}

model = GridSearchCV(pipe, param_grid=param_grid, scoring='r2', verbose=50)

predictions = np.empty([data_features_test.index.unique().size,4])
predictions.shape

best_scores = []
m = 0
for vital in vitals:

    model.fit(X_3_train, data_labels_train[vital])
    predictions[:,m] = model.predict(X_3_test) 
    best_scores.append(model.best_score_)
    m = m+1

print(f'The mean r2-score is: {np.mean(best_scores)}')


Fitting 5 folds for each of 4 candidates, totalling 20 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] regr=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False), regr__max_features=sqrt, regr__n_estimators=500 


KeyboardInterrupt: ignored

In [ ]:

predictions_fixed = predictions[:,0:4]

In [ ]:
submission_3 = pd.DataFrame(predictions_fixed, index=data_features_test.index.unique(), columns=data_labels_train[vitals].columns)
submission_3.head

In [60]:
#import previously calculated values
old_prediction = pd.read_csv('/content/drive/MyDrive/ml_task2/prediction.zip', index_col='pid')

old_prediction_subtask3 = old_prediction[vitals]
submission = pd.concat((df_12, old_prediction_subtask3), axis=1)
submission.to_csv('prediction.zip', index=True, float_format='%.3f', compression='zip')
submission.to_csv('prediction.csv', index=True, float_format='%.3f')